# CFPB API Client Testing Notebook

This notebook demonstrates how to use the CFPBAPIClient to pull consumer complaint data from the CFPB API and convert it to pandas DataFrames for analysis.

## Sections:
1. **Setup & Imports**: Import required libraries
2. **Class Definition**: CFPBAPIClient class
3. **Basic Testing**: Test API connection
4. **Data Exploration**: Convert to DataFrame and explore
5. **Advanced Queries**: Filter and analyze data


In [15]:
# Import required libraries
import logging
from typing import Optional, Dict, Any, List
from datetime import datetime, timedelta
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import json

# Setup logging for better visibility
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## 1. CFPBAPIClient Class Definition

This class handles all interactions with the CFPB Consumer Complaint Database API.

In [16]:
class CFPBAPIClient:
    """Client for accessing the CFPB Consumer Complaint Database API."""
    
    BASE_URL = "https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/"
    DEFAULT_TIMEOUT = 30
    MAX_RETRIES = 3
    
    def __init__(self, timeout: int = DEFAULT_TIMEOUT):
        """Initialize the CFPB API client."""
        self.timeout = timeout
        self.session = self._create_session()
        
    def _create_session(self) -> requests.Session:
        """Create a requests session with retry logic and proper headers."""
        session = requests.Session()
        
        # CRITICAL FIX: Add User-Agent header (CFPB API requires this!)
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (compatible; ConsumerComplaintETL/1.0; Python/requests)',
            'Accept': 'application/json',
        })
        
        retry_strategy = Retry(
            total=self.MAX_RETRIES,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504],
            allowed_methods=["GET"]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)
        session.mount("http://", adapter)
        return session
    
    def get_complaints(
        self,
        date_received_min: Optional[str] = None,
        date_received_max: Optional[str] = None,
        size: int = 10000,
        frm: int = 0,
        sort: str = "created_date_desc",
        search_term: Optional[str] = None,
        search_field: Optional[str] = None,
        no_aggs: bool = False,
        **filters
    ) -> Dict[str, Any]:
        """Fetch consumer complaints from the CFPB API with better error handling."""
        params = {
            "size": min(size, 10000),
            "frm": frm,
            "sort": sort,
            "format": "json"
        }
        
        if date_received_min:
            params["date_received_min"] = date_received_min
        if date_received_max:
            params["date_received_max"] = date_received_max
            
        # Add search parameters for company-specific queries
        if search_term:
            params["search_term"] = search_term
        if search_field:
            params["field"] = search_field
        if no_aggs:
            params["no_aggs"] = "true"
            
        params.update(filters)
        
        try:
            print(f"Requesting: {self.BASE_URL}")
            print(f"Params: {params}")
            
            response = self.session.get(self.BASE_URL, params=params, timeout=self.timeout)
            
            print(f"Status Code: {response.status_code}")
            
            if response.status_code != 200:
                print(f"ERROR - Response: {response.text[:500]}")
            
            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error: {e}")
            print(f"Response: {e.response.text[:500] if e.response else 'No response'}")
            raise
        except Exception as e:
            print(f"Unexpected error: {type(e).__name__}: {e}")
            raise
    
    def get_complaints_list(
        self,
        date_received_min: Optional[str] = None,
        date_received_max: Optional[str] = None,
        max_records: Optional[int] = None,
        **filters
    ) -> List[Dict[str, Any]]:
        """Fetch complaints and return as a list of dictionaries."""
        response = self.get_complaints(
            date_received_min=date_received_min,
            date_received_max=date_received_max,
            size=max_records or 1000,
            **filters
        )
        
        # Handle different response formats
        if isinstance(response, list):
            # Direct list format (newer API response)
            complaints = [hit.get("_source", {}) for hit in response]
        elif isinstance(response, dict) and 'hits' in response:
            # Nested dict format (older API response)
            hits = response.get("hits", {}).get("hits", [])
            complaints = [hit.get("_source", {}) for hit in hits]
        else:
            complaints = []
            
        return complaints
    
    def get_complaints_last_n_days(self, days: int = 7, max_records: Optional[int] = None) -> List[Dict[str, Any]]:
        """Fetch complaints from the last N days."""
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        return self.get_complaints_list(
            date_received_min=start_date.strftime("%Y-%m-%d"),
            date_received_max=end_date.strftime("%Y-%m-%d"),
            max_records=max_records
        )
    
    def get_complaints_by_company(
        self,
        company_name: str,
        start_date: Optional[str] = None,
        end_date: Optional[str] = None,
        max_records: Optional[int] = None,
        no_aggs: bool = True
    ) -> List[Dict[str, Any]]:
        """
        Fetch complaints for a specific company.
        
        Args:
            company_name: Name of the company to search for (e.g., 'jpmorgan')
            start_date: Start date (YYYY-MM-DD), defaults to '2011-12-01'
            end_date: End date (YYYY-MM-DD), defaults to today
            max_records: Maximum total records to fetch (None for all available)
            no_aggs: Disable aggregations for faster responses (default: True)
            
        Returns:
            List of complaint records for the specified company
        """
        # Set default dates if not provided
        if not start_date:
            start_date = '2011-12-01'  # CFPB database start date
        if not end_date:
            end_date = datetime.now().strftime('%Y-%m-%d')
            
        print(f"Searching for complaints from {company_name}")
        print(f"Date range: {start_date} to {end_date}")
        
        response = self.get_complaints(
            date_received_min=start_date,
            date_received_max=end_date,
            search_term=company_name,
            search_field='company',
            no_aggs=no_aggs,
            size=max_records or 10000
        )
        
        # Handle different response formats
        if isinstance(response, list):
            # Direct list format (newer API response)
            complaints = [hit.get("_source", {}) for hit in response]
            total_available = len(complaints)
        elif isinstance(response, dict) and 'hits' in response:
            # Nested dict format (older API response)
            hits = response.get("hits", {}).get("hits", [])
            complaints = [hit.get("_source", {}) for hit in hits]
            total_value = response.get("hits", {}).get("total", {})
            if isinstance(total_value, dict):
                total_available = total_value.get("value", 0)
            else:
                total_available = total_value
        else:
            complaints = []
            total_available = 0
        
        print(f"Total complaints available: {total_available:,}")
        print(f"Fetched: {len(complaints)} complaints")
        
        return complaints
    
    def close(self):
        """Close the API client session."""
        if self.session:
            self.session.close()

print("CFPBAPIClient class defined successfully!")

CFPBAPIClient class defined successfully!


## 2. Initialize the API Client

Create an instance of the CFPBAPIClient to start making API requests.

In [17]:
# Initialize the client
client = CFPBAPIClient()
print("CFPB API Client initialized successfully!")
print(f"Base URL: {client.BASE_URL}")

CFPB API Client initialized successfully!
Base URL: https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/


## 3. Debug API Connection
First, let's test the API endpoint directly to see what's happening.

In [18]:
# Test the API directly with company search + User-Agent header (FIXED!)
import requests

test_url = "https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/"

# Test with small dataset: JPMorgan complaints from Jan 2024
test_params = {
    "search_term": "jpmorgan",           # Search for specific company
    "field": "company",                  # Search in company field
    "date_received_min": "2024-01-01",   # Start date
    "date_received_max": "2024-01-31",   # End date (just January for small test)
    "no_aggs": "true",                   # Disable aggregations for faster response
    "size": 10,                          # Small sample for testing
    "format": "json"
}

# CRITICAL FIX: Add User-Agent header!
headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; ConsumerComplaintETL/1.0; Python/requests)',
    'Accept': 'application/json',
}

print("Testing API endpoint with company search and date filters...")
print(f"URL: {test_url}")
print(f"\nQuery Parameters:")
print(f"  Company: {test_params['search_term']}")
print(f"  Date Range: {test_params['date_received_min']} to {test_params['date_received_max']}")
print(f"  Sample Size: {test_params['size']}")
print(f"\nHeaders: {headers}")
print("\n" + "="*80)

try:
    # THIS IS THE FIX - pass headers parameter!
    response = requests.get(test_url, params=test_params, headers=headers, timeout=30)
    print(f"Response Status: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print(f"\nAPI is working!")
        
        # Handle different response formats
        hits = []
        total = 0
        
        if isinstance(data, list):
            # Direct list format (newer API response)
            print(f"\nResponse format: Direct list")
            hits = data
            total = len(hits)
            print(f"Fetched: {len(hits)} complaints")
            
        elif isinstance(data, dict) and 'hits' in data:
            # Nested dict format (older API response)
            print(f"\nResponse format: Nested dictionary")
            total_value = data.get('hits', {}).get('total', {})
            if isinstance(total_value, dict):
                total = total_value.get('value', 0)
            else:
                total = total_value
            print(f"Total complaints available: {total:,}")
            
            hits = data.get('hits', {}).get('hits', [])
            print(f"Fetched: {len(hits)} complaints")
        else:
            print(f"\nWARNING: Unexpected response format")
            print(f"Type: {type(data)}")
            print(f"Preview: {str(data)[:500]}")
        
        # Show sample complaints
        if hits:
            print(f"\n{'='*80}")
            print("Sample Complaints:")
            print('='*80)
            
            # Show up to 3 sample complaints
            for i, hit in enumerate(hits[:3], 1):
                source = hit.get('_source', {})
                print(f"\n{i}. Complaint ID: {hit.get('_id', 'N/A')}")
                print(f"   Company: {source.get('company', 'N/A')}")
                print(f"   Date Received: {source.get('date_received', 'N/A')}")
                print(f"   Product: {source.get('product', 'N/A')}")
                print(f"   Issue: {source.get('issue', 'N/A')}")
                print(f"   State: {source.get('state', 'N/A')}")
                
                # Show snippet of complaint text if available
                complaint_text = source.get('complaint_what_happened', '')
                if complaint_text:
                    snippet = complaint_text[:150] + '...' if len(complaint_text) > 150 else complaint_text
                    print(f"   Description: {snippet}")
            
            print(f"\n{'='*80}")
    else:
        print(f"\nERROR: HTTP {response.status_code}")
        print(f"Response text: {response.text[:500]}")
        
except Exception as e:
    print(f"ERROR: {type(e).__name__}")
    print(f"Error message: {str(e)}")
    import traceback
    print(f"\nFull traceback:")
    traceback.print_exc()
    print(f"\nPossible issues:")
    print("  - Unexpected API response format")
    print("  - Network connectivity")
    print("  - API endpoint changed")
    print("  - Rate limiting")
    print("  - Firewall/proxy blocking the request")
    
print("\n" + "="*80)
print("NOTE: Adjust the parameters above to test different companies or date ranges")

Testing API endpoint with company search and date filters...
URL: https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/

Query Parameters:
  Company: jpmorgan
  Date Range: 2024-01-01 to 2024-01-31
  Sample Size: 10

Headers: {'User-Agent': 'Mozilla/5.0 (compatible; ConsumerComplaintETL/1.0; Python/requests)', 'Accept': 'application/json'}

Response Status: 200

API is working!

Response format: Direct list
Fetched: 1504 complaints

Sample Complaints:

1. Complaint ID: 8141559
   Company: JPMORGAN CHASE & CO.
   Date Received: 2024-01-11T12:00:00-05:00
   Product: Checking or savings account
   Issue: Managing an account
   State: CA
   Description: Chase Bank allowed fraudulent charges with my debit card then took back the money my account was credited when I called to report the theft of my card...

2. Complaint ID: 8198984
   Company: JPMORGAN CHASE & CO.
   Date Received: 2024-01-22T12:00:00-05:00
   Product: Checking or savings account
   Issue: Managing

In [19]:
# Convert API response to DataFrame
print("Converting API response to pandas DataFrame...")

if 'hits' in locals() and hits:
    # Extract the _source data from each hit
    complaints_list = [hit.get('_source', {}) for hit in hits]
    df_test = pd.DataFrame(complaints_list)
    
    print(f"DataFrame created successfully!")
    print(f"   Shape: {df_test.shape} (rows, columns)")
    print(f"   Columns: {df_test.shape[1]}")
    
    # Show available columns
    print(f"\nAvailable columns:")
    for col in sorted(df_test.columns):
        print(f"   - {col}")
    
    # Display DataFrame preview with key columns
    print(f"\nDataFrame Preview:")
    key_cols = ['date_received', 'company', 'product', 'issue', 'state']
    # Only show columns that exist
    display_cols = [col for col in key_cols if col in df_test.columns]
    display(df_test[display_cols].head())
    
    print(f"\nDataFrame saved as 'df_test' variable for further analysis")
else:
    print("WARNING: No data available. Please run the cell above first to fetch data from the API.")


Converting API response to pandas DataFrame...
DataFrame created successfully!
   Shape: (1504, 18) (rows, columns)
   Columns: 18

Available columns:
   - company
   - company_public_response
   - company_response
   - complaint_id
   - complaint_what_happened
   - consumer_consent_provided
   - consumer_disputed
   - date_received
   - date_sent_to_company
   - issue
   - product
   - state
   - sub_issue
   - sub_product
   - submitted_via
   - tags
   - timely
   - zip_code

DataFrame Preview:


,date_received,company,product,issue,state
0,2024-01-11T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Managing an account,CA
1,2024-01-22T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Managing an account,FL
2,2024-01-23T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Closing an account,NC
3,2024-01-04T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit card,Getting a credit card,PA
4,2024-01-29T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Managing an account,NY



DataFrame saved as 'df_test' variable for further analysis


## 4. Convert to DataFrame

Now let's fetch data and convert it to a pandas DataFrame for easier analysis.

In [20]:
# Convert API response to DataFrame
print("Converting API response to pandas DataFrame...")

if 'hits' in locals() and hits:
    # Extract the _source data from each hit
    complaints_list = [hit.get('_source', {}) for hit in hits]
    df_test = pd.DataFrame(complaints_list)
    
    print(f"DataFrame created successfully!")
    print(f"   Shape: {df_test.shape} (rows, columns)")
    print(f"   Columns: {df_test.shape[1]}")
    
    # Show available columns
    print(f"\nAvailable columns:")
    for col in sorted(df_test.columns):
        print(f"   - {col}")
    
    # Display DataFrame preview with key columns
    print(f"\nDataFrame Preview:")
    key_cols = ['date_received', 'company', 'product', 'issue', 'state']
    # Only show columns that exist
    display_cols = [col for col in key_cols if col in df_test.columns]
    display(df_test[display_cols].head())
    
    print(f"\nDataFrame saved as 'df_test' variable for further analysis")
else:
    print("WARNING: No data available. Please run the cell above first to fetch data from the API.")

Converting API response to pandas DataFrame...
DataFrame created successfully!
   Shape: (1504, 18) (rows, columns)
   Columns: 18

Available columns:
   - company
   - company_public_response
   - company_response
   - complaint_id
   - complaint_what_happened
   - consumer_consent_provided
   - consumer_disputed
   - date_received
   - date_sent_to_company
   - issue
   - product
   - state
   - sub_issue
   - sub_product
   - submitted_via
   - tags
   - timely
   - zip_code

DataFrame Preview:


,date_received,company,product,issue,state
0,2024-01-11T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Managing an account,CA
1,2024-01-22T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Managing an account,FL
2,2024-01-23T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Closing an account,NC
3,2024-01-04T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit card,Getting a credit card,PA
4,2024-01-29T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Managing an account,NY



DataFrame saved as 'df_test' variable for further analysis


## 5. Data Exploration

Let's explore the data to understand what we're working with.


In [21]:
# View column information
print("DataFrame Columns and Data Types:")
print("="*80)
df_test.info()

print("\n" + "="*80)
print("\n Summary Statistics:")
df_test.describe(include='all')


DataFrame Columns and Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1504 entries, 0 to 1503
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   product                    1504 non-null   object
 1   complaint_what_happened    1504 non-null   object
 2   date_sent_to_company       1504 non-null   object
 3   issue                      1504 non-null   object
 4   sub_product                1504 non-null   object
 5   zip_code                   1504 non-null   object
 6   tags                       195 non-null    object
 7   complaint_id               1504 non-null   object
 8   timely                     1504 non-null   object
 9   consumer_consent_provided  1504 non-null   object
 10  company_response           1504 non-null   object
 11  submitted_via              1504 non-null   object
 12  company                    1504 non-null   object
 13  date_received              15

,product,complaint_what_happened,date_sent_to_company,issue,sub_product,zip_code,tags,complaint_id,timely,consumer_consent_provided,company_response,submitted_via,company,date_received,state,consumer_disputed,company_public_response,sub_issue
count,1504,1504,1504,1504,1504,1504,195,1504,1504,1504,1504,1504,1504,1504,1485,1504,0,1369
unique,10,787,42,53,31,1169,3,1504,1,5,3,4,1,31,49,1,0,108
top,Checking or savings account,,2024-01-18T12:00:00-05:00,Managing an account,Checking account,XXXXX,Older American,8141559,Yes,Consent provided,Closed with explanation,Web,JPMORGAN CHASE & CO.,2024-01-24T12:00:00-05:00,CA,N/A,NaN,Deposits and withdrawals
freq,563,716,88,328,511,62,94,1,1504,789,1224,1332,1504,88,267,1504,NaN,131


In [22]:
# Check for missing values
print(" Missing Values Analysis:")
print("="*80)
missing_data = df_test.isnull().sum()
missing_pct = (df_test.isnull().sum() / len(df_test)) * 100

missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing Count': missing_data.values,
    'Missing %': missing_pct.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
 print(" No missing values found!")

 Missing Values Analysis:
                 Column  Missing Count  Missing %
company_public_response           1504 100.000000
                   tags           1309  87.034574
              sub_issue            135   8.976064
                  state             19   1.263298


In [23]:
# Analyze key columns
print(" Top 10 Products by Complaint Count:")
print("="*80)
if 'product' in df_test.columns:
    product_counts = df_test['product'].value_counts().head(10)
    print(product_counts)
    print(f"\nTotal unique products: {df_test['product'].nunique()}")

print("\n" + "="*80)
print("\n Top 10 Companies by Complaint Count:")
if 'company' in df_test.columns:
    company_counts = df_test['company'].value_counts().head(10)
    print(company_counts)
    print(f"\nTotal unique companies: {df_test['company'].nunique()}")


 Top 10 Products by Complaint Count:
product
Checking or savings account                                563
Credit card                                                450
Credit reporting or other personal consumer reports        261
Money transfer, virtual currency, or money service         128
Debt collection                                             45
Mortgage                                                    29
Vehicle loan or lease                                       20
Payday loan, title loan, personal loan, or advance loan      6
Debt or credit management                                    1
Student loan                                                 1
Name: count, dtype: int64

Total unique products: 10


 Top 10 Companies by Complaint Count:
company
JPMORGAN CHASE & CO.    1504
Name: count, dtype: int64

Total unique companies: 1


## 6. Advanced Queries

Test different query parameters and filters.

In [24]:
# Test: Fetch complaints for a specific date range
print(" Fetching complaints for a specific date range...")

start_date = "2024-01-01"
end_date = "2024-01-31"

date_range_data = client.get_complaints_list(
    date_received_min=start_date,
    date_received_max=end_date,
    max_records=50  # Limit for testing
)

df_date_range = pd.DataFrame(date_range_data)

print(f" Fetched {len(df_date_range)} complaints for {start_date} to {end_date}")
print(f" DataFrame shape: {df_date_range.shape}")
print("\n Sample data:")
df_date_range.head()


 Fetching complaints for a specific date range...
Requesting: https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/
Params: {'size': 50, 'frm': 0, 'sort': 'created_date_desc', 'format': 'json', 'date_received_min': '2024-01-01', 'date_received_max': '2024-01-31'}
Status Code: 200
 Fetched 143378 complaints for 2024-01-01 to 2024-01-31
 DataFrame shape: (143378, 18)

 Sample data:


,product,complaint_what_happened,date_sent_to_company,issue,sub_product,zip_code,tags,complaint_id,timely,consumer_consent_provided,company_response,submitted_via,company,date_received,state,consumer_disputed,company_public_response,sub_issue
0,Credit card,,2024-01-29T12:00:00-05:00,Getting a credit card,General-purpose credit card or charge card,62703,None,8247641,Yes,Consent not provided,Closed with explanation,Web,AMERICAN EXPRESS COMPANY,2024-01-29T12:00:00-05:00,IL,N/A,None,Application denied
1,Credit reporting or other personal consumer re...,,2024-01-26T12:00:00-05:00,Problem with a company's investigation into an...,Credit reporting,30097,None,8229724,Yes,Consent not provided,Closed with non-monetary relief,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2024-01-26T12:00:00-05:00,GA,N/A,Company has responded to the consumer and the ...,Their investigation did not fix an error on yo...
2,Credit reporting or other personal consumer re...,,2024-01-10T12:00:00-05:00,Improper use of your report,Credit reporting,75126,None,8137583,Yes,Consent not provided,Closed with non-monetary relief,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2024-01-10T12:00:00-05:00,TX,N/A,Company has responded to the consumer and the ...,Reporting company used your report improperly
3,Credit reporting or other personal consumer re...,"This inquiry in not mine, I don't have any rel...",2024-01-09T12:00:00-05:00,Incorrect information on your report,Credit reporting,XXXXX,None,8134929,Yes,Consent provided,Closed with explanation,Web,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",2024-01-09T12:00:00-05:00,AZ,N/A,Company has responded to the consumer and the ...,Information belongs to someone else
4,Credit reporting or other personal consumer re...,,2024-01-20T12:00:00-05:00,Incorrect information on your report,Credit reporting,33321,None,8187016,Yes,Consent not provided,Closed with non-monetary relief,Web,Experian Information Solutions Inc.,2024-01-20T12:00:00-05:00,FL,N/A,Company has responded to the consumer and the ...,Information belongs to someone else


In [28]:
# Test: Filter by specific product
print(" Fetching complaints for a specific product...")

# Example: Credit card complaints
product_data = client.get_complaints_list(
    product="Credit card",
    max_records=50
)

if product_data:
    df_product = pd.DataFrame(product_data)
    print(f" Fetched {len(df_product)} Credit card complaints")
    print(f"\n Issues breakdown:")
    if 'issue' in df_product.columns:
        print(df_product['issue'].value_counts().head(10))
else:
    print("WARNING: No data returned for this product")

 Fetching complaints for a specific product...
Requesting: https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/
Params: {'size': 50, 'frm': 0, 'sort': 'created_date_desc', 'format': 'json', 'product': 'Credit card'}
Status Code: 200
 Fetched 254910 Credit card complaints

 Issues breakdown:
issue
Problem with a purchase shown on your statement                    32650
Incorrect information on your report                               25310
Problem with a company's investigation into an existing problem    23856
Getting a credit card                                              21015
Other features, terms, or problems                                 15521
Billing disputes                                                   15136
Fees or interest                                                   13576
Closing your account                                                9892
Other                                                               9353
Problem when maki

## 6.3. Search by Company Name

Test the new company search functionality with customizable date ranges.


In [27]:
# Example: Fetch complaints for JPMorgan Chase
print(" Fetching complaints for a specific company...")
print("="*80)

# Search for JPMorgan complaints
company_data = client.get_complaints_by_company(
    company_name='jpmorgan',
    start_date='2024-01-01',  # Optional: specify start date
    end_date='2025-12-31',    # Optional: specify end date
    max_records=100           # Limit for testing
)

if company_data:
    df_company = pd.DataFrame(company_data)
    print(f"\n Successfully fetched {len(df_company)} complaints")
    print(f" DataFrame shape: {df_company.shape}")
    
    # Show company distribution (should be mostly JPMorgan variants)
    print(f"\n Company name variations found:")
    print(df_company['company'].value_counts().head(10))
    
    # Show products breakdown
    if 'product' in df_company.columns:
        print(f"\n Products breakdown:")
        print(df_company['product'].value_counts().head(10))
    
    # Show issues breakdown
    if 'issue' in df_company.columns:
        print(f"\nWARNING: Issues breakdown:")
        print(df_company['issue'].value_counts().head(10))
    
    # Display sample data
    print(f"\n Sample data:")
    display(df_company[['date_received', 'company', 'product', 'issue', 'state']].head(10))
else:
    print("WARNING: No data returned for this company")

print("\n" + "="*80)
print("NOTE: You can search for any company name:")
print("   - Use lowercase for better matching (e.g., 'chase', 'bank of america')")
print("   - Adjust start_date and end_date to filter by date range")
print("   - Remove max_records limit to fetch all available complaints")

 Fetching complaints for a specific company...
Searching for complaints from jpmorgan
Date range: 2024-01-01 to 2025-12-31
Requesting: https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/
Params: {'size': 100, 'frm': 0, 'sort': 'created_date_desc', 'format': 'json', 'date_received_min': '2024-01-01', 'date_received_max': '2025-12-31', 'search_term': 'jpmorgan', 'field': 'company', 'no_aggs': 'true'}
Status Code: 200
Total complaints available: 36,629
Fetched: 36629 complaints

 Successfully fetched 36629 complaints
 DataFrame shape: (36629, 18)

 Company name variations found:
company
JPMORGAN CHASE & CO.    36629
Name: count, dtype: int64

 Products breakdown:
product
Checking or savings account                                13441
Credit card                                                 9476
Credit reporting or other personal consumer reports         7522
Money transfer, virtual currency, or money service          2936
Debt collection                   

,date_received,company,product,issue,state
0,2025-06-05T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Closing an account,NY
1,2025-01-16T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit card,Problem with a purchase shown on your statement,VA
2,2025-06-26T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit card,"Other features, terms, or problems",NY
3,2024-02-02T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Managing an account,NJ
4,2024-10-31T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit card,Problem with a purchase shown on your statement,CA
5,2025-01-16T12:00:00-05:00,JPMORGAN CHASE & CO.,"Money transfer, virtual currency, or money ser...",Other transaction problem,CA
6,2024-10-07T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit card,"Other features, terms, or problems",CT
7,2025-04-04T12:00:00-05:00,JPMORGAN CHASE & CO.,Checking or savings account,Closing an account,GA
8,2025-09-13T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit reporting or other personal consumer re...,Incorrect information on your report,MD
9,2025-09-19T12:00:00-05:00,JPMORGAN CHASE & CO.,Credit card,Incorrect information on your report,CA



NOTE: You can search for any company name:
   - Use lowercase for better matching (e.g., 'chase', 'bank of america')
   - Adjust start_date and end_date to filter by date range
   - Remove max_records limit to fetch all available complaints


## 7. Export Data

Save the DataFrame to various formats for further analysis.


In [ ]:
df_company.to_csv('../data/jpmorgan_complaints.csv', index=False)
print("Data saved to jpmorgan_complaints.csv")

print("NOTE: Uncomment the export lines above to save data to files")

Data saved to jpmorgan_complaints.csv
💡 Uncomment the export lines above to save data to files


## 8. Cleanup

Close the API client connection when done.

In [ ]:
# Close the API client connection
client.close()
print(" API client connection closed")
print("\n" + "="*80)
print(" Notebook execution complete!")
print("="*80)

✅ API client connection closed

🎉 Notebook execution complete!


## Summary

This notebook demonstrated how to:

1. ✅ **Initialize** the CFPB API client
2. ✅ **Fetch data** from the CFPB Consumer Complaint Database API
3. ✅ **Convert** API responses to pandas DataFrames
4. ✅ **Explore** the data structure and content
5. ✅ **Filter** data by date ranges and products
6. ✅ **Analyze** complaint patterns and trends
7. ✅ **Export** data to various formats

## Key Takeaways

- The API returns data in a nested JSON structure under `hits.hits._source`
- Each complaint contains ~20+ fields including product, company, issue, etc.
- The API supports filtering by date, product, company, state, and more
- Maximum 10,000 records per API call (use pagination for more)
- Data can be easily converted to pandas DataFrame for analysis

## Next Steps

1. **Increase Data Volume**: Remove the `max_records` limit to fetch more data
2. **Add Filters**: Experiment with state, company, or issue filters
3. **Time Series Analysis**: Analyze complaint trends over time
4. **Data Visualization**: Create charts and graphs using matplotlib/seaborn
5. **Load to Snowflake**: Use the main ETL pipeline to load data to Snowflake

## API Documentation

For more details, see: https://cfpb.github.io/api/ccdb/api.html